## iQuantum Dataset Generator

In [1]:
import os
import pandas as pd
from qiskit import QuantumCircuit
from qiskit.qasm import Qasm

def analyze_qasm(qasm_path):
    with open(qasm_path, "r") as qasm_file:
        qasm_str = qasm_file.read()
    qc = QuantumCircuit.from_qasm_str(qasm_str)
    n_qubits = qc.num_qubits
    qc_depth = qc.depth()

    single_qubit_gate_names = ["u1", "u2", "u3", "id", "x", "y", "z", "h", "s", "sdg", "t", "tdg", "rx", "ry", "rz", "sx", "measure"]
    multi_qubit_gate_names = ["cx", "ccx"]

    single_qubit_gates = {}
    multi_qubit_gates = {}

    gate_counts = qc.count_ops()

    for gate, count in gate_counts.items():
        if gate in single_qubit_gate_names:
            single_qubit_gates[gate] = count
        elif gate in multi_qubit_gate_names:
            multi_qubit_gates[gate] = count

    topology = set()
    cx_instruction = qc.get_instructions(name="cx")
    for instr, qargs, _ in cx_instruction:
            qubit_pair = tuple([qarg.index for qarg in qargs])  # Sort qubit indices to ensure uniqueness
            topology.add(qubit_pair)
            topology.add(qubit_pair[::-1])  # Add the opposite connection
    topology_sorted = sorted(list(topology))
    topology_str = "; ".join(map(str, topology_sorted))

    return {
        "n_qubits": n_qubits,
        "qc_depth": qc_depth,
        "single_qubit_gates": single_qubit_gates,
        "multi_qubit_gates": multi_qubit_gates,
        "qubit_topology": topology_str,
    }
    
def export_to_csv(data, csv_path):
    df = pd.DataFrame(data)
    df.index.name = "index"
    df.to_csv(csv_path, index=True)

In [4]:
folder_path = "QASM/MQT-Set1-298-10-27-IBMQ27-Opt3"
csv_output_path = "MQT-Set1-298-10-27-IBMQ27-Opt3.csv"

qasm_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith(".qasm")]

analyzed_data = []

for qasm_path in qasm_files:
    # print(f"Processing: {qasm_path}")  # Add this line to print the file being processed
    analysis = analyze_qasm(qasm_path)
    analysis["qasm_file"] = os.path.basename(qasm_path)
    analyzed_data.append(analysis)

export_to_csv(analyzed_data, csv_output_path)

/var/folders/_s/jg1zw33138n0bcr554kbw2lm0000gn/T/ipykernel_59609/2019718186.py:30: DeprecationWarning: Back-references to from Bit instances to their containing Registers have been deprecated. Instead, inspect Registers to find their contained Bits.
  qubit_pair = tuple([qarg.index for qarg in qargs])  # Sort qubit indices to ensure uniqueness
